Based on: https://github.dev/huggingface/notebooks/blob/main/sagemaker/23_stable_diffusion_inference/sagemaker-notebook.ipynb

TODO: Incooperate multi modal endpoint: https://www.philschmid.de/stable-diffusion-inpainting-inference-endpoints

In [29]:
import sagemaker.huggingface
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

True

## Permissions

In [30]:
import sagemaker
import boto3

ACCESS_KEY = os.environ.get("AWS_SAGEMAKER_ACCESS_KEY")
SECRET_KEY = os.environ.get("AWS_SAGEMAKER_SECRET_ACCESS_KEY")
REGION = os.environ.get("AWS_BUCKET_REGION")

iam_client = boto3.client('iam', aws_access_key_id=ACCESS_KEY,
                          aws_secret_access_key=SECRET_KEY)

boto_session = boto3.Session(aws_access_key_id=ACCESS_KEY,
                             aws_secret_access_key=SECRET_KEY, region_name=REGION)

role = iam_client.get_role(
    RoleName='sagemaker')['Role']['Arn']
sess = sagemaker.Session(boto_session=boto_session)

### Util functions

In [31]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import matplotlib.pyplot as plt
import io

# helper decoder
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = BytesIO(base64_image)
  return Image.open(buffer)

def encode_base64_image(image):
  buffered = BytesIO()
  image.save(buffered, format="JPEG")
  return base64.b64encode(buffered.getvalue()).decode()

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)

def image_from_s3(bucket, key, s3):
  bucket = s3.Bucket(bucket)
  image = bucket.Object(key)
  img_data = image.get().get('Body').read()
  return Image.open(io.BytesIO(img_data))

In [ ]:
from sagemaker.huggingface import HuggingFaceModel

# User id will be provided by request later
USER_ID = "ztvuzveD7nZRRFYCgc9gvnagTwr2"
USER_GENDER = "woman"

# Store training results
MODEL_BUCKET = "sagemaker-user-training"
# model_path = f's3://{MODEL_BUCKET}/{USER_ID}/model.tar.gz'
model_path = f"s3://{MODEL_BUCKET}/{USER_ID}/ztvuzveD7nZRRFYCgc9gvnagTwr2-2022-12-30-15-06-02-633/output/model.tar.gz"

# Store instance images as train images and class images as test images
IMAGES_BUCKET = "sagemaker-user-io"
instance_input_path = f's3://{IMAGES_BUCKET}/{USER_ID}/'

# TODO: Path to example application picture which is used as starting point for inference.
# Will be stored somewhere in f's3://{IMAGES_BUCKET}/{USER_ID}/...'
#instance_input_key = f"{USER_ID}/140e9145-5d97-4796-a94c-24283960d800.JPG"

#s3 = boto3.resource('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)

# Download the image
#instance_input_image = image_from_s3(IMAGES_BUCKET, instance_input_key, s3)

from IPython.display import display
import requests
from PIL import Image


url = "https://ca.slack-edge.com/TUTJQ5RNV-U02P1ET637V-7d91d73a4fd4-512"
instance_input_image = Image.open(requests.get(url, stream=True).raw)
display(instance_input_image)

In [35]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=model_path,
   entry_point='inference.py',
   source_dir="./inference",
   role=role,
   transformers_version='4.17',
   pytorch_version='1.10',
   py_version='py38',
)

In [36]:
# We can also add autoscaling
# https://www.philschmid.de/auto-scaling-sagemaker-huggingface

# deploy the endpoint endpoint
# We can also deploy for serverless inference: 
# https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-estimator
# Also eslatic inference could be interesting
# accelerator_type (str) – The Elastic Inference accelerator type to deploy to the instance for loading and making inferences to the model.
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge"
    )

----------------!

In [38]:
prompt = f"Professional portrait photo of qonvhs {USER_GENDER} in formal clothing, perfect eyes, studio lighting, HDR, UHD, 4K, 8k, 64K, highly detailed, portrait, 40mm lens, shallow depth of field, close up, studio lighting",
negative_prompt = "blender, closed eyes, jewlery, chain,ugly, different eye color, multiple hands, bad anatomy, bad proportions, unrealistic, full body, cropped, lowres, poorly drawn face, out of frame, poorly drawn hands, double, blurred, disfigured, deformed, repetitive, black and white",
init_image_base64 = encode_base64_image(instance_input_image)     # Has to be the image we use as reference. Encoded to base64 for request
strength = 0.825            # Regulates how "creative" the model is
num_images_per_prompt = 5  # How many images to generate per request
num_inference_steps = 70    # Depends on denoiser we choose. But in general 50 for DDIM is good
guidance_scale = 9.0        # How much the model follows the text prompt
SEED = 52362

# run prediction
response = predictor.predict(data={
    # Check inference.py for parameters it expects
    "pos_prompt": prompt,
    "neg_prompt": negative_prompt,
    "depth_image_base64": init_image_base64,
    "strength": strength,
    "num_images_per_prompt": num_images_per_prompt,
    "num_inference_steps": num_inference_steps,
    "guidance_scale": guidance_scale,
    "seed": SEED,
  }
)

# TODO: Decode application image to base64 -> Allows to send the request to backend
decoded_images = [decode_base64_image(image) for image in response["generated_images"]]

# visualize generation
display_images(decoded_images)

In [27]:
predictor.delete_model()

In [28]:
predictor.delete_endpoint()